In [370]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [371]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import ast
import json

In [372]:

# Load your data (replace with your file path)
#data= pd.read_parquet('your_landmark_data.parquet')

df_from_parquet = pd.read_parquet(r"C:\Users\ranjan.patra\OneDrive - Lingaro Sp. z o. o\DATA\IITJ\Course\Projects\ISL\ISL_CSLRT_Corpus\Mp_Data\LandMarks_lh_rh")


In [373]:
unique_labels = df_from_parquet['Label'].unique()
label_id_mapping = {label: idx for idx, label in enumerate(unique_labels)}
# Step 3: Map the Label to Label_id in the DataFrame
df_from_parquet['Label_id'] = df_from_parquet['Label'].map(label_id_mapping)
label_info = [{label, label_id} for label, label_id in label_id_mapping.items()]

In [374]:
df_1=df_from_parquet[["Label_id","Sample_Number","Frame_Number","Keypoints"]]

In [375]:
def flatten_keypoints(keypoints):
    # Check if the keypoints is a string or an array
    if isinstance(keypoints, str):
        # If it's a string, remove the curly braces and split into individual components
        keypoints_str = keypoints.strip('{}')
        keypoints_list = keypoints_str.split(',')
        # Convert to float and return as a numpy array
        return np.array([float(kp) for kp in keypoints_list])
    elif isinstance(keypoints, np.ndarray):
        # If it's already an ndarray, flatten it directly
        flattened = np.concatenate([np.array([float(kp) for kp in kp_str.strip('{}').split(',')]) for kp_str in keypoints])
        return flattened
    else:
        return np.array([])  # Return an empty array for unexpected types

# Apply the flattening function to the Keypoints column
df_1['keypoints_flattened'] = df_1['Keypoints'].apply(flatten_keypoints)

# Display the DataFrame with the new column
#print(df_1[['Label_id', 'Sample_Number', 'Frame_Number', 'keypoints_flattened']])


C:\Users\ranjan.patra\AppData\Local\Temp\ipykernel_13148\3559514141.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['keypoints_flattened'] = df_1['Keypoints'].apply(flatten_keypoints)


import pandas as pd
import numpy as np

# Assuming df_1 is already defined and contains columns: 'Label_id', 'Sample_Number', 'Frame_Number', 'Keypoints'

# Step 1: Group by Label_id and Sample_Number
grouped = df_1.groupby(['Label_id', 'Sample_Number'])

# Step 2: Create a list to hold the sequences
sequences = []

# Define the required number of timesteps (n_timesteps = 192 in this case)
unique_frame_numbers = df_1['Frame_Number'].unique()
n_timesteps = len(unique_frame_numbers) 

for (label_id, sample_number), group in grouped:
    keypoints = group['keypoints_flattened'].to_list()  # Extract keypoints as a list of arrays
    
    # If the number of frames is less than the required n_timesteps
    if len(keypoints) < n_timesteps:
        # Get the last keypoint (to use for padding)
        last_keypoint = keypoints[-1]
        # Add the last keypoint repeatedly until we reach 192 frames
        while len(keypoints) < n_timesteps:
            keypoints.append(last_keypoint)

    # Now keypoints has exactly 192 frames
    for i in range(len(keypoints) - n_timesteps + 1):
        sequence = keypoints[i:i + n_timesteps]
        sequences.append((label_id, sample_number, sequence))

# Step 3: Convert to numpy array
X = np.array([seq[2] for seq in sequences])  # Extract sequences
y = np.array([seq[0] for seq in sequences])  # Extract labels (Label_id)

# Reshape X for LSTM (samples, timesteps, features)
# Assuming each keypoint array is of size (75,)
X = X.reshape((X.shape[0], n_timesteps, 225))

# Now X is ready for LSTM model input
print(f'X shape: {X.shape}, y shape: {y.shape}')


In [406]:
import pandas as pd
import numpy as np

# Assuming df_1 is already defined and contains columns: 'Label_id', 'Sample_Number', 'Frame_Number', 'Keypoints'

# Step 1: Group by Label_id and Sample_Number
grouped = df_1.groupby(['Label_id', 'Sample_Number'])

# Step 2: Create a list to hold the sequences
sequences = []

# Define the required number of timesteps (n_timesteps = 192 in this case)
unique_frame_numbers = df_1['Frame_Number'].unique()
n_timesteps = len(unique_frame_numbers) 

for (label_id, sample_number), group in grouped:
    keypoints = group['keypoints_flattened'].to_list()  # Extract keypoints as a list of arrays
        
    # If the number of frames is less than the required n_timesteps
    if len(keypoints) < n_timesteps:
        # Get the last keypoint (to use for padding)
        last_keypoint = keypoints[-1]
        # Add the last keypoint repeatedly until we reach 192 frames
        while len(keypoints) < n_timesteps:
            keypoints.append(last_keypoint)

    # Flatten keypoints (assuming each keypoint is an array of shape (75,))
    flattened_keypoints = [kp.flatten() for kp in keypoints]  # Flatten each keypoint

    # Now keypoints has exactly 192 frames
    for i in range(len(flattened_keypoints) - n_timesteps + 1):
        sequence = flattened_keypoints[i:i + n_timesteps]
        sequences.append((label_id, sample_number, sequence))

# Step 3: Convert to numpy array
X = np.array([seq[2] for seq in sequences])  # Extract sequences
y = np.array([seq[0] for seq in sequences])  # Extract labels (Label_id)

# Reshape X for LSTM (samples, timesteps, features)
# Assuming each keypoint array is of size (75,), it will remain (75,) even after flattening
X = X.reshape((X.shape[0], n_timesteps, 225))

# Now X is ready for LSTM model input
print(f'X shape: {X.shape}, y shape: {y.shape}')


X shape: (487, 192, 225), y shape: (487,)


In [407]:
np.array(X).shape


(487, 192, 225)

In [408]:
y.shape

(487,)

In [409]:
y = to_categorical(y).astype(int)


In [410]:
y.shape

(487, 100)

In [411]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10)


In [412]:
y_test.shape

(49, 100)

In [421]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam
import os

In [422]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [436]:
y.shape[1]

100

In [437]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(192,225)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))

In [438]:
res = [.7, 0.2, 0.1]

In [440]:
y[np.argmax(res)]

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [441]:
model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
optimizer = Adam(clipvalue=1.0)

In [442]:
model.fit(X, y, epochs=200, callbacks=[tb_callback])


Epoch 1/200
16/16 [==============================] - 8s 330ms/step - loss: 4.6059 - categorical_accuracy: 0.0021
Epoch 2/200
16/16 [==============================] - 5s 327ms/step - loss: 4.6056 - categorical_accuracy: 0.0041
Epoch 3/200
16/16 [==============================] - 6s 386ms/step - loss: 4.6054 - categorical_accuracy: 0.0062
Epoch 4/200
16/16 [==============================] - 6s 393ms/step - loss: 4.6052 - categorical_accuracy: 0.0062
Epoch 5/200
16/16 [==============================] - 6s 401ms/step - loss: 4.6050 - categorical_accuracy: 0.0062
Epoch 6/200
16/16 [==============================] - 6s 383ms/step - loss: 4.6049 - categorical_accuracy: 0.0062
Epoch 7/200
16/16 [==============================] - 6s 401ms/step - loss: 4.6047 - categorical_accuracy: 0.0062
Epoch 8/200
16/16 [==============================] - 6s 365ms/step - loss: 4.6047 - categorical_accuracy: 0.0062
Epoch 9/200
16/16 [==============================] - 6s 358ms/step - loss: 4.6045 - categorical_

In [443]:
model.summary()


Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_64 (LSTM)              (None, 192, 64)           74240     
                                                                 
 lstm_65 (LSTM)              (None, 192, 128)          98816     
                                                                 
 lstm_66 (LSTM)              (None, 64)                49408     
                                                                 
 dense_63 (Dense)            (None, 64)                4160      
                                                                 
 dense_64 (Dense)            (None, 32)                2080      
                                                                 
 dense_65 (Dense)            (None, 100)               3300      
                                                                 
Total params: 232,004
Trainable params: 232,004
Non-t

In [444]:
res = model.predict(X_test)


2/2 [==============================] - 1s 205ms/step


In [ ]:
X_test

In [369]:
y[np.argmax(res[4])]


array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [449]:
import torch
from model import T2M_GPT


ModuleNotFoundError: No module named 'model'